In [1]:
import pandas as pd
import numpy as np
import time
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

## School Progress Report (SPR) Data

In [36]:
spr_yoy = pd.read_excel('data/SPR_SY1617_Data/SPR_YearoverYear_20180129.xlsx', \
                             sheet_name="Sheet1", skiprows=9, header=0)

spr_yoy['School ID'] = spr_yoy['School ID'].astype(str)
spr_yoy['School ID_old'] = spr_yoy['School ID_old'].astype(str)
spr_yoy = spr_yoy[(spr_yoy["2017-18 Learning Network"] != "Closed")]

print(spr_yoy.shape)
spr_yoy.head(3)

(321, 33)


,Unnamed: 0,School ID_old,Dup_id_ind,School ID,School Name,2017-18 Learning Network,Report Type (School Level),YOY Change (2015-16 to 2016-17),2016-2017 Achievement Score,2016-2017 Progress Score,2016-2017 Climate Score,2016-2017 College&Career Score,2016-2017 Overall Score,2015-2016 Achievement Score,2015-2016 Progress Score,2015-2016 Climate Score,2015-2016 College&Career Score,2015-2016 Overall Score,2014-2015 Achievement Score,2014-2015 Progress Score,2014-2015 Climate Score,2014-2015 College&Career Score,2014-2015 Overall Score,2013-2014 Achievement Score,2013-2014 Progress Score,2013-2014 Climate Score,2013-2014 College&Career Score,2013-2014 Overall Score,2012-2013 Achievement Score,2012-2013 Progress Score,2012-2013 Climate Score,2012-2013 College&Career Score,2012-2013 Overall Score
0,241HS,241,1,241HS,Girard Academic Music Program,Autonomy Network,HS,7.0,56.0,73.0,77.0,77.0,69,52.0,49.0,86.0,83.0,62,54.0,61.0,67.0,91.0,64,64.0,67.0,93.0,98.0,76,74,59,100,97,77
1,241MS,241,1,241MS,Girard Academic Music Program,Autonomy Network,MS,-15.0,84.0,46.0,93.0,NaN,72,79.0,90.0,92.0,NaN,87,71.0,51.0,90.0,NaN,69,84.0,32.0,99.0,NaN,62,83,40,100,NaN,65
2,403HS,403,1,403HS,High School of Engineering and Science,Autonomy Network,HS,15.0,71.0,78.0,80.0,77.0,76,58.0,45.0,90.0,68.0,61,56.0,60.0,81.0,85.0,66,66.0,84.0,94.0,95.0,82,70,90,97,98,87


In [37]:
spr_yoy.tail(7)

,Unnamed: 0,School ID_old,Dup_id_ind,School ID,School Name,2017-18 Learning Network,Report Type (School Level),YOY Change (2015-16 to 2016-17),2016-2017 Achievement Score,2016-2017 Progress Score,2016-2017 Climate Score,2016-2017 College&Career Score,2016-2017 Overall Score,2015-2016 Achievement Score,2015-2016 Progress Score,2015-2016 Climate Score,2015-2016 College&Career Score,2015-2016 Overall Score,2014-2015 Achievement Score,2014-2015 Progress Score,2014-2015 Climate Score,2014-2015 College&Career Score,2014-2015 Overall Score,2013-2014 Achievement Score,2013-2014 Progress Score,2013-2014 Climate Score,2013-2014 College&Career Score,2013-2014 Overall Score,2012-2013 Achievement Score,2012-2013 Progress Score,2012-2013 Climate Score,2012-2013 College&Career Score,2012-2013 Overall Score
362,X24K8,X24,0,X24,Universal Charter School at Alcorn,Charters,K8,12.0,15.0,83.0,62.0,NaN,56,5.0,66.0,54.0,NaN,44,3.0,76.0,25.0,NaN,37,3.0,12.0,48.0,NaN,17,NaN,NaN,NaN,NaN,Not Eligible
364,X26K8,X26,0,X26,Mastery Charter School at Pastorius,Charters,K8,-23.0,8.0,29.0,30.0,NaN,23,10.0,84.0,34.0,NaN,46,10.0,50.0,41.0,NaN,35,22.0,84.0,52.0,NaN,59,NaN,NaN,NaN,NaN,Not Eligible
365,X27ES,X27,0,X27,Mastery Charter School at Wister*,Charters,ES,NaN,7.0,51.0,27.0,NaN,31,NaN,NaN,NaN,NaN,Not Eligible,NaN,NaN,NaN,NaN,Not Eligible,NaN,NaN,NaN,NaN,Not Eligible,NaN,NaN,NaN,NaN,Not Eligible
366,X28K8,X28,0,X28,GLA Charter at Huey*,Charters,K8,NaN,1.0,22.0,49.0,NaN,24,NaN,NaN,NaN,NaN,Not Eligible,NaN,NaN,NaN,NaN,Not Eligible,NaN,NaN,NaN,NaN,Not Eligible,NaN,NaN,NaN,NaN,Not Eligible
367,X40ES,X40,0,X40,MaST Community Charter II*,Charters,ES,NaN,56.0,NaN,93.0,NaN,75,NaN,NaN,NaN,NaN,Not Eligible,NaN,NaN,NaN,NaN,Not Eligible,NaN,NaN,NaN,NaN,Not Eligible,NaN,NaN,NaN,NaN,Not Eligible
368,X41HS,X41,0,X41,TECH Freire Charter School*,Charters,HS,NaN,1.0,28.0,12.0,NaN,15,NaN,NaN,NaN,NaN,Not Eligible,NaN,NaN,NaN,NaN,Not Eligible,NaN,NaN,NaN,NaN,Not Eligible,NaN,NaN,NaN,NaN,Not Eligible
369,X42ES,X42,0,X42,Independence Charter School West*,Charters,ES,NaN,20.0,NaN,88.0,NaN,57,NaN,NaN,NaN,NaN,Not Eligible,NaN,NaN,NaN,NaN,Not Eligible,NaN,NaN,NaN,NaN,Not Eligible,NaN,NaN,NaN,NaN,Not Eligible


In [38]:
def latestScores(row, score_type):
    year1 = str('2016-2017 ') + str(score_type)
    year2 = str('2015-2016 ') + str(score_type)
    year3 = str('2014-2015 ') + str(score_type)
    year4 = str('2013-2014 ') + str(score_type)
    year5 = str('2012-2013 ') + str(score_type)

    if not pd.isnull(row[year1]):
        score = row[year1]
    elif not pd.isnull(row[year2]):
        score = row[year2]
    elif not pd.isnull(row[year3]):
        score = row[year3]
    elif not pd.isnull(row[year4]):
        score = row[year4]
    elif not pd.isnull(row[year5]):
        score = row[year5]
    else:
        score = np.nan
        
    return score

spr_yoy['Achievement_Score_Final'] = spr_yoy.apply(latestScores, score_type='Achievement Score', axis=1)
spr_yoy['Progress_Score_Final'] = spr_yoy.apply(latestScores, score_type='Progress Score', axis=1)
spr_yoy['Climate_Score_Final'] = spr_yoy.apply(latestScores, score_type='Climate Score', axis=1)
spr_yoy['Overall_Score_Final'] = spr_yoy.apply(latestScores, score_type='Overall Score', axis=1)

In [39]:
# read SPR data
spr = pd.read_excel('data/SPR_SY1617_Data/SPR_SY1617_School_Metric_Scores_20180206.xlsx', sheet_name=1)

# convert School ID to string
spr = spr.rename(columns={'SRC School ID': 'School ID'})
spr['School ID'] = spr['School ID'].astype(str)
#spr['School ID_old'] = spr['School ID'].astype(str)
# spr.set_index(['School ID', 'Report'], inplace=True)

spr['Full Address'] = spr['Street Address'] + ', ' + spr['City'] + ', ' + spr['State'] + ' ' + spr['Zip Code'].astype(str)
spr.drop(['Street Address', 'City', 'State', 'Zip Code', 'Phone Number', 'Fax Number'], axis=1, inplace=True)
spr.head(3)

,School,SRC School ID OLD,Dup_ID_ind,School ID,Report,Rpt Type Long,Governance,Turnaround Model,Enrollment,Website,Grades Served,Admissions Type,2016-17 Learning Network,2017-18 Learning Network,Notes,Overall Score,Overall Tier,Overall Peer Rank,Overall City Rank,Ach Score,Ach Tier,Ach Peer Rank,Ach City Rank,Prog Score,Prog Tier,Prog Peer Rank,Prog City Rank,Clim Score,Clim Tier,Clim Peer Rank,Clim City Rank,CC Score,CC Tier,CC Peer Rank,CC City Rank,K-2 Rdg Score,K-2 Rdg Denom,K-2 Rdg Pts Earn,K-2 Rdg Pts Poss,K-2 Rdg Pct Earn,K-2 Rdg Tier,PSSA ELA Prof/Adv Score,PSSA ELA Prof/Adv Denom,PSSA ELA Prof/Adv Pts Earn,PSSA ELA Prof/Adv Pts Poss,PSSA ELA Prof/Adv Pct Earn,PSSA ELA Prof/Adv Tier,PSSA GR3 Prof/Adv ELA Score,PSSA GR3 Prof/Adv ELA Denom,PSSA GR4-8 Prof/Adv ELA Score,PSSA GR4-8 Prof/Adv ELA Denom,PSSA ELA Adv Score,PSSA ELA Adv Denom,PSSA ELA Adv Pts Earn,PSSA ELA Adv Pts Poss,PSSA ELA Adv Pct Earn,PSSA ELA Adv Tier,PSSA GR3 Adv ELA Score,PSSA GR3 Adv ELA Denom,PSSA GR4-8 Adv ELA Score,PSSA GR4-8 Adv ELA Denom,PSSA Math Prof/Adv Score,PSSA Math Prof/Adv Denom,PSSA Math Prof/Adv Pts Earn,PSSA Math Prof/Adv Pts Poss,PSSA Math Prof/Adv Pct Earn,PSSA Math Prof/Adv Tier,PSSA Math Adv Score,PSSA Math Adv Denom,PSSA Math Adv Pts Earn,PSSA Math Adv Pts Poss,PSSA Math Adv Pct Earn,PSSA Math Adv Tier,PSSA Sci Prof/Adv Score,PSSA Sci Prof/Adv Denom,PSSA Sci Prof/Adv Pts Earn,PSSA Sci Prof/Adv Pts Poss,PSSA Sci Prof/Adv Pct Earn,PSSA Sci Prof/Adv Tier,PSSA Sci Adv Score,PSSA Sci Adv Denom,PSSA Sci Adv Pts Earn,PSSA Sci Adv Pts Poss,PSSA Sci Adv Pct Earn,PSSA Sci Adv Tier,Keystone Alg1 Prof/Adv Score,Keystone Alg1 Prof/Adv Denom,Keystone Alg1 Prof/Adv Pts Earn,Keystone Alg1 Prof/Adv Pts Poss,Keystone Alg1 Prof/Adv Pct Earn,Keystone Alg1 Prof/Adv Tier,Keystone Alg1 Adv Score,Keystone Alg1 Adv Denom,Keystone Alg1 Adv Pts Earn,Keystone Alg1 Adv Pts Poss,Keystone Alg1 Adv Pct Earn,Keystone Alg1 Adv Tier,Keystone Bio Prof/Adv Score,Keystone Bio Prof/Adv Denom,Keystone Bio Prof/Adv Pts Earn,Keystone Bio Prof/Adv Pts Poss,Keystone Bio Prof/Adv Pct Earn,Keystone Bio Prof/Adv Tier,Keystone Bio Adv Score,Keystone Bio Adv Denom,Keystone Bio Adv Pts Earn,Keystone Bio Adv Pts Poss,Keystone Bio Adv Pct Earn,Keystone Bio Adv Tier,Keystone Lit Prof/Adv Score,Keystone Lit Prof/Adv Denom,Keystone Lit Prof/Adv Pts Earn,Keystone Lit Prof/Adv Pts Poss,Keystone Lit Prof/Adv Pct Earn,Keystone Lit Prof/Adv Tier,Keystone Lit Adv Score,Keystone Lit Adv Denom,Keystone Lit Adv Pts Earn,Keystone Lit Adv Pts Poss,Keystone Lit Adv Pct Earn,Keystone Lit Adv Tier,ACCESS Prof Score,ACCESS Prof Denom,ACCESS Prof Pts Earn,ACCESS Prof Pts Poss,ACCESS Prof Pct Earn,ACCESS Prof Tier,PSSA Math AGI Score,PSSA Math AGI Pts Earn,PSSA Math AGI Pts Poss,PSSA Math AGI Pct Earn,PSSA Math AGI Tier,PSSA ELA AGI Score,PSSA ELA AGI Pts Earn,PSSA ELA AGI Pts Poss,PSSA ELA AGI Pct Earn,PSSA ELA AGI Tier,PSSA GR4 Sci AGI Score,PSSA GR4 Sci AGI Pts Earn,PSSA GR4 Sci AGI Pts Poss,PSSA GR4 Sci AGI Pct Earn,PSSA GR4 Sci AGI Tier,PSSA GR8 Sci AGI Score,PSSA GR8 Sci AGI Pts Earn,PSSA GR8 Sci AGI Pts Poss,PSSA GR8 Sci AGI Pct Earn,PSSA GR8 Sci AGI Tier,Keystone Alg1 AGI Score,Keystone Alg1 AGI Pts Earn,Keystone Alg1 AGI Pts Poss,Keystone Alg1 AGI Pct Earn,Keystone Alg1 AGI Tier,Keystone Bio AGI Score,Keystone Bio AGI Pts Earn,Keystone Bio AGI Pts Poss,Keystone Bio AGI Pct Earn,Keystone Bio AGI Tier,Keystone Lit AGI Score,Keystone Lit AGI Pts Earn,Keystone Lit AGI Pts Poss,Keystone Lit AGI Pct Earn,Keystone Lit AGI Tier,Stay On Track Score,Stay On Track Denom,Stay On Track Pts Earn,Stay On Track Pts Poss,Stay On Track Pct Earn,Stay On Track Tier,Back On Track Score,Back On Track Denom,Back On Track Pts Earn,Back On Track Pts Poss,Back On Track Pct Earn,Back On Track Tier,ACCESS Growth Score,ACCESS Growth Denom,ACCESS Growth Pts Earn,ACCESS Growth Pts Poss,ACCESS Growth Pct Earn,ACCESS Growth Tier,PSSA Math AGI (L20P) Score,PSSA Math AGI (L20P) Pts Earn,PSSA Math AGI (L20P) Pts Poss,PSSA Math 

# Get GPS Coordinates
For each school, use the address to find the Latitude and Longitude Coordinates. This will let us place the schools on a map.

In [40]:
geolocator = Nominatim()
cell_start = time.time()

lat = pd.Series({})
lon = pd.Series({})

for index, row in spr.iterrows():
    addr = row['Full Address']

    if row['School ID'] not in lat:
        # try geocoder 4 times
        for i in range(0,4):
            try:
                location = geolocator.geocode(addr)
                break
            except GeocoderTimedOut:
                print("Geocoder Timed Out. Address =", addr)
                continue

        try:
            latitude = location.latitude
            longitude = location.longitude
        except AttributeError:
            latitude = np.nan
            longitude = np.nan

        lat = pd.concat([lat, pd.Series({row['School ID']: latitude})])
        lon = pd.concat([lon, pd.Series({row['School ID']: longitude})])
    
    else:
        continue
        
print('Cell Runtime:', round(time.time() - cell_start), 'seconds')

Geocoder Timed Out. Address = 5400 Warrington Ave, Philadelphia, PA 19143
Geocoder Timed Out. Address = 4021 Parkside Ave, Philadelphia, PA 19104
Geocoder Timed Out. Address = 301 N 36th St, Philadelphia, PA 19104
Geocoder Timed Out. Address = 2000 N 7th St, Philadelphia, PA 19122
Geocoder Timed Out. Address = 3500 Academy Rd, Philadelphia, PA 19154
Geocoder Timed Out. Address = 1023 Callowhill St, Philadelphia, PA 19123
Cell Runtime: 267 seconds


In [41]:
print(lat.shape, sum(pd.isnull(lat)))
lat = lat.rename("Latitude")
lon = lon.rename("Longitude")

(321,) 51


In [42]:
df = pd.merge(spr, spr_yoy, how='inner', on='School ID')
df = pd.merge(df, lat.to_frame(), how='left', left_on='School ID', right_index=True)
df = pd.merge(df, lon.to_frame(), how='left', left_on='School ID', right_index=True)

df = df[['School','School ID_old', 'School ID', 'Full Address', 'Enrollment', 'Rpt Type Long','Governance', 'Achievement_Score_Final', \
         'Progress_Score_Final', 'Climate_Score_Final', 'Overall_Score_Final', 'Latitude', 'Longitude']]
df.head()

,School,School ID_old,School ID,Full Address,Enrollment,Rpt Type Long,Governance,Achievement_Score_Final,Progress_Score_Final,Climate_Score_Final,Overall_Score_Final,Latitude,Longitude
0,Girard Academic Music Program,241,241HS,"2136 W Ritner St, Philadelphia, PA 19145",239,High School,District,56.0,73.0,77.0,69,39.922118,-75.18261
1,Girard Academic Music Program,241,241MS,"2136 W Ritner St, Philadelphia, PA 19145",333,Middle School,District,84.0,46.0,93.0,72,39.922118,-75.18261
2,High School of Engineering and Science,403,403HS,"1600 W Norris St, Philadelphia, PA 19121",776,High School,District,71.0,78.0,80.0,76,NaN,NaN
3,High School of Engineering and Science,403,403MS,"1600 W Norris St, Philadelphia, PA 19121",125,Middle School,District,58.0,19.0,94.0,54,NaN,NaN
4,Penn Treaty High School,516,516HS,"600 E Thompson St, Philadelphia, PA 19125",346,High School,District,3.0,39.0,14.0,19,NaN,NaN


In [43]:
type(df['School ID_old'][1])


str

## Serious Incidents (aka Safety)

In [26]:
si_1314 = pd.read_csv('data/Serious_Incidents/Serious_Incidents_2013-14.TXT')
si_1415 = pd.read_excel('data/Serious_Incidents/Serious_Incidents_2014-15.xlsx', sheet_name=0)
si_1415.rename(columns={'ULCS Code':'ULCS_NO', 
                        'School Year':'SCHOOL_YEAR', 
                        'Incident Type':'INCIDENT_TYPE',
                        'Incident Count':'INCIDENT_COUNT',
                        'School ID':'SCHOOL_ID'}, inplace=True)

si_1516 = pd.read_excel('data/Serious_Incidents/Serious_Incidents_2015-16.xls', sheet_name=0)
si_1617 = pd.read_excel('data/Serious_Incidents/Serious_Incidents_2016-17.xlsx', sheet_name=0)

si = pd.concat([si_1314, si_1415, si_1516, si_1617]).reset_index(drop=True)
si.drop('ULCS_NO', axis=1, inplace=True)
si['INCIDENT_TYPE'] = si['INCIDENT_TYPE'].str.upper()
si['INCIDENT_TYPE'].replace('SUICIDE - ATTEMPTS & THREATS', 'SUICIDE - ATTEMPTS AND THREATS', inplace=True)
si['INCIDENT_TYPE'].replace('DRUGS & ALCOHOL OFFENSE', 'DRUG AND ALCOHOL OFFENSES', inplace=True)
si['INCIDENT_TYPE'].replace('DRUG & ALCOHOL OFFENSES', 'DRUG AND ALCOHOL OFFENSES', inplace=True)
si['INCIDENT_TYPE'].replace('THREATS', 'THREAT', inplace=True)
si['INCIDENT_TYPE'].replace('THEFTS', 'THEFT', inplace=True)
si['INCIDENT_TYPE'].replace('BOMB SCARES', 'BOMB SCARE', inplace=True)
si['INCIDENT_TYPE'].replace('FIRES AND FALSE ALARMS', 'FIRE - INCENDIARY', inplace=True)
si['INCIDENT_TYPE'].replace('THREATS - VERBAL AND WRITTEN', 'THREAT', inplace=True)
si['INCIDENT_TYPE'].replace('WEAPONS OFFENSES', 'WEAPONS', inplace=True)

# convert School ID to string
si['SCHOOL_ID'] = si['SCHOOL_ID'].astype(str)

si.head()

,INCIDENT_COUNT,INCIDENT_TYPE,SCHOOL_ID,SCHOOL_YEAR
0,1,HARASSMENT,526,2013-2014
1,5,ASSAULTS,534,2013-2014
2,8,THREAT,535,2013-2014
3,0,TAKE OFF,537,2013-2014
4,0,DRUG AND ALCOHOL OFFENSES,545,2013-2014


In [44]:
si2 = pd.pivot_table(si, values='INCIDENT_COUNT', 
                   index='SCHOOL_ID', 
                   columns=['INCIDENT_TYPE'], 
                   aggfunc=np.sum).fillna(0).reset_index()

# convert School ID to string
si2['SCHOOL_ID'] = si2['SCHOOL_ID'].astype(str)


pd.set_option('display.max_columns', 500)
df2 = pd.merge(df, si2, how='left', left_on='School ID_old', right_on='SCHOOL_ID')
df2.drop('SCHOOL_ID', axis=1, inplace=True)

In [18]:
# df2.head(3)
# si.head(3)

In [45]:
for val in si['INCIDENT_TYPE'].unique():
    df2[val + '_rate'] = (df2[val] * 100) / (df2['Enrollment'] * 4)
    
df2.head()

,School,School ID_old,School ID,Full Address,Enrollment,Rpt Type Long,Governance,Achievement_Score_Final,Progress_Score_Final,Climate_Score_Final,Overall_Score_Final,Latitude,Longitude,ASSAULTS,BOMB SCARE,BULLYING,BURGLARY,DISORDERLY CONDUCT,DRUG AND ALCOHOL OFFENSES,FIRE - INCENDIARY,GRAFFITI,HARASSMENT,MORALS OFFENSES,ROBBERY,SUICIDE,SUICIDE - ATTEMPTS AND THREATS,TAKE OFF,THEFT,THREAT,TRESPASSING,VANDALISM,WEAPONS,HARASSMENT_rate,ASSAULTS_rate,THREAT_rate,TAKE OFF_rate,DRUG AND ALCOHOL OFFENSES_rate,BURGLARY_rate,TRESPASSING_rate,BULLYING_rate,WEAPONS_rate,BOMB SCARE_rate,FIRE - INCENDIARY_rate,ROBBERY_rate,THEFT_rate,VANDALISM_rate,DISORDERLY CONDUCT_rate,SUICIDE_rate,MORALS OFFENSES_rate,GRAFFITI_rate,SUICIDE - ATTEMPTS AND THREATS_rate
0,Girard Academic Music Program,241,241HS,"2136 W Ritner St, Philadelphia, PA 19145",239,High School,District,56.0,73.0,77.0,69,39.922118,-75.18261,3.0,0.0,0.0,1.0,4.0,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,3.0,0.000000,0.313808,0.104603,0.0,0.104603,0.104603,0.000000,0.0,0.313808,0.000000,0.104603,0.209205,0.523013,0.00000,0.418410,0.000000,0.104603,0.000000,0.0
1,Girard Academic Music Program,241,241MS,"2136 W Ritner St, Philadelphia, PA 19145",333,Middle School,District,84.0,46.0,93.0,72,39.922118,-75.18261,3.0,0.0,0.0,1.0,4.0,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,3.0,0.000000,0.225225,0.075075,0.0,0.075075,0.075075,0.000000,0.0,0.225225,0.000000,0.075075,0.150150,0.375375,0.00000,0.300300,0.000000,0.075075,0.000000,0.0
2,High School of Engineering and Science,403,403HS,"1600 W Norris St, Philadelphia, PA 19121",776,High School,District,71.0,78.0,80.0,76,NaN,NaN,3.0,1.0,0.0,0.0,11.0,1.0,0.0,0.0,3.0,3.0,2.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,3.0,0.096649,0.096649,0.064433,0.0,0.032216,0.000000,0.000000,0.0,0.096649,0.032216,0.000000,0.064433,0.128866,0.00000,0.354381,0.000000,0.096649,0.000000,0.0
3,High School of Engineering and Science,403,403MS,"1600 W Norris St, Philadelphia, PA 19121",125,Middle School,District,58.0,19.0,94.0,54,NaN,NaN,3.0,1.0,0.0,0.0,11.0,1.0,0.0,0.0,3.0,3.0,2.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,3.0,0.600000,0.600000,0.400000,0.0,0.200000,0.000000,0.000000,0.0,0.600000,0.200000,0.000000,0.400000,0.800000,0.00000,2.200000,0.000000,0.600000,0.000000,0.0
4,Penn Treaty High School,516,516HS,"600 E Thompson St, Philadelphia, PA 19125",346,High School,District,3.0,39.0,14.0,19,NaN,NaN,90.0,0.0,0.0,0.0,100.0,27.0,1.0,1.0,10.0,17.0,9.0,2.0,0.0,0.0,7.0,37.0,3.0,7.0,29.0,0.722543,6.502890,2.673410,0.0,1.950867,0.000000,0.216763,0.0,2.095376,0.000000,0.072254,0.650289,0.505780,0.50578,7.225434,0.144509,1.228324,0.072254,0.0


In [47]:
from sklearn import preprocessing

# scale 'recipient_cfscore' to be 0-1 so it matches our ideology score above
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,100))

df2['SafetyIncidentRate'] = df2['ASSAULTS_rate'] + df2['BULLYING_rate'] + df2['HARASSMENT_rate'] + \
                        df2['BURGLARY_rate'] + df2['ROBBERY_rate'] + df2['THEFT_rate'] + df2['WEAPONS_rate'] + \
                        df2['SUICIDE_rate']

null_index = df2['SafetyIncidentRate'].isnull() 
df2.loc[~null_index, 'SafetyScore'] = 100 - min_max_scaler.fit_transform(np.array(df2.loc[~null_index, 'SafetyIncidentRate']).reshape(-1, 1))

df2.SafetyScore = df2.SafetyScore.round()


df2.head()

,School,School ID_old,School ID,Full Address,Enrollment,Rpt Type Long,Governance,Achievement_Score_Final,Progress_Score_Final,Climate_Score_Final,Overall_Score_Final,Latitude,Longitude,ASSAULTS,BOMB SCARE,BULLYING,BURGLARY,DISORDERLY CONDUCT,DRUG AND ALCOHOL OFFENSES,FIRE - INCENDIARY,GRAFFITI,HARASSMENT,MORALS OFFENSES,ROBBERY,SUICIDE,SUICIDE - ATTEMPTS AND THREATS,TAKE OFF,THEFT,THREAT,TRESPASSING,VANDALISM,WEAPONS,HARASSMENT_rate,ASSAULTS_rate,THREAT_rate,TAKE OFF_rate,DRUG AND ALCOHOL OFFENSES_rate,BURGLARY_rate,TRESPASSING_rate,BULLYING_rate,WEAPONS_rate,BOMB SCARE_rate,FIRE - INCENDIARY_rate,ROBBERY_rate,THEFT_rate,VANDALISM_rate,DISORDERLY CONDUCT_rate,SUICIDE_rate,MORALS OFFENSES_rate,GRAFFITI_rate,SUICIDE - ATTEMPTS AND THREATS_rate,SafetyIncidentRate,SafetyScore
0,Girard Academic Music Program,241,241HS,"2136 W Ritner St, Philadelphia, PA 19145",239,High School,District,56.0,73.0,77.0,69,39.922118,-75.18261,3.0,0.0,0.0,1.0,4.0,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,3.0,0.000000,0.313808,0.104603,0.0,0.104603,0.104603,0.000000,0.0,0.313808,0.000000,0.104603,0.209205,0.523013,0.00000,0.418410,0.000000,0.104603,0.000000,0.0,1.464435,91.0
1,Girard Academic Music Program,241,241MS,"2136 W Ritner St, Philadelphia, PA 19145",333,Middle School,District,84.0,46.0,93.0,72,39.922118,-75.18261,3.0,0.0,0.0,1.0,4.0,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,5.0,1.0,0.0,0.0,3.0,0.000000,0.225225,0.075075,0.0,0.075075,0.075075,0.000000,0.0,0.225225,0.000000,0.075075,0.150150,0.375375,0.00000,0.300300,0.000000,0.075075,0.000000,0.0,1.051051,94.0
2,High School of Engineering and Science,403,403HS,"1600 W Norris St, Philadelphia, PA 19121",776,High School,District,71.0,78.0,80.0,76,NaN,NaN,3.0,1.0,0.0,0.0,11.0,1.0,0.0,0.0,3.0,3.0,2.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,3.0,0.096649,0.096649,0.064433,0.0,0.032216,0.000000,0.000000,0.0,0.096649,0.032216,0.000000,0.064433,0.128866,0.00000,0.354381,0.000000,0.096649,0.000000,0.0,0.483247,97.0
3,High School of Engineering and Science,403,403MS,"1600 W Norris St, Philadelphia, PA 19121",125,Middle School,District,58.0,19.0,94.0,54,NaN,NaN,3.0,1.0,0.0,0.0,11.0,1.0,0.0,0.0,3.0,3.0,2.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,3.0,0.600000,0.600000,0.400000,0.0,0.200000,0.000000,0.000000,0.0,0.600000,0.200000,0.000000,0.400000,0.800000,0.00000,2.200000,0.000000,0.600000,0.000000,0.0,3.000000,82.0
4,Penn Treaty High School,516,516HS,"600 E Thompson St, Philadelphia, PA 19125",346,High School,District,3.0,39.0,14.0,19,NaN,NaN,90.0,0.0,0.0,0.0,100.0,27.0,1.0,1.0,10.0,17.0,9.0,2.0,0.0,0.0,7.0,37.0,3.0,7.0,29.0,0.722543,6.502890,2.673410,0.0,1.950867,0.000000,0.216763,0.0,2.095376,0.000000,0.072254,0.650289,0.505780,0.50578,7.225434,0.144509,1.228324,0.072254,0.0,10.621387,38.0


In [48]:
df2.tail()

,School,School ID_old,School ID,Full Address,Enrollment,Rpt Type Long,Governance,Achievement_Score_Final,Progress_Score_Final,Climate_Score_Final,Overall_Score_Final,Latitude,Longitude,ASSAULTS,BOMB SCARE,BULLYING,BURGLARY,DISORDERLY CONDUCT,DRUG AND ALCOHOL OFFENSES,FIRE - INCENDIARY,GRAFFITI,HARASSMENT,MORALS OFFENSES,ROBBERY,SUICIDE,SUICIDE - ATTEMPTS AND THREATS,TAKE OFF,THEFT,THREAT,TRESPASSING,VANDALISM,WEAPONS,HARASSMENT_rate,ASSAULTS_rate,THREAT_rate,TAKE OFF_rate,DRUG AND ALCOHOL OFFENSES_rate,BURGLARY_rate,TRESPASSING_rate,BULLYING_rate,WEAPONS_rate,BOMB SCARE_rate,FIRE - INCENDIARY_rate,ROBBERY_rate,THEFT_rate,VANDALISM_rate,DISORDERLY CONDUCT_rate,SUICIDE_rate,MORALS OFFENSES_rate,GRAFFITI_rate,SUICIDE - ATTEMPTS AND THREATS_rate,SafetyIncidentRate,SafetyScore
308,Mastery Charter School at Wister,X27,X27,"67 E Bringhurst St, Philadelphia, PA 19144",515,Elementary School,Charter,7.0,51.0,27.0,31,40.032274,-75.167095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309,Global Leadership Academy at Huey,X28,X28,"5200 Pine St, Philadelphia, PA 19143",511,K8 School,Charter,1.0,22.0,49.0,24,39.953465,-75.226398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
310,MaST Community Charter School II,X40,X40,"6238 Rising Sun Ave, Philadelphia, PA 19111",394,Elementary School,Charter,56.0,NaN,93.0,75,40.047399,-75.097868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
311,TECH Freire Charter School,X41,X41,"2221 N Broad St, Philadelphia, PA 19132",304,High School,Charter,1.0,28.0,12.0,15,39.986667,-75.156084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
312,Independence Charter School West,X42,X42,"5600 Chester Ave, Philadelphia, PA 19143",300,Elementary School,Charter,20.0,NaN,88.0,57,39.937247,-75.226226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Special Education Data

In [50]:
df_iep= pd.read_csv('Special-Education/IEP_data/final_IED.csv')
df_iep.head()

,Unnamed: 0,School_ID,School_Code,School Name_x,Cohort,Subgroup,Students,Grads,Rate,Governance,Unnamed: 8,Unnamed: 9,Unnamed: 10,Test Name,Subject,School Name_y,Learning Network,Grade,Category,Number Tested,Below Basic Count #,Below Basic Percent %,Basic Count #,Basic Percent %,Proficient Count #,Proficient Basic Percent %,Advanced Count #,Advanced Percent %,Prof and Adv Count #,Prof and Adv. Percent %,Average Score,Average Score2
0,0,101,101,JOHN BARTRAM HIGH SCHOOL,2013-2014,IEP,53,22,42%,District,NaN,NaN,NaN,Keystone,Algebra 1,John Bartram High School,NEIGHBORHOOD NETWORK 1,11,IEP,31,23,74%,8,26%,0,0%,0,0%,0,0%,0.314516129,1.258064508
1,1,101,101,JOHN BARTRAM HIGH SCHOOL,2013-2014,IEP,53,22,42%,District,NaN,NaN,NaN,Keystone,Algebra 1,John Bartram High School,NEIGHBORHOOD NETWORK 1,11,Not IEP,63,30,48%,27,43%,6,10%,0,0%,6,10%,0.404761905,1.619047634
2,2,101,101,JOHN BARTRAM HIGH SCHOOL,2013-2014,IEP,53,22,42%,District,NaN,NaN,NaN,Keystone,Biology,John Bartram High School,NEIGHBORHOOD NETWORK 1,11,IEP,28,24,86%,4,14%,0,0%,0,0%,0,0%,0.285714286,1.142857164
3,3,101,101,JOHN BARTRAM HIGH SCHOOL,2013-2014,IEP,53,22,42%,District,NaN,NaN,NaN,Keystone,Biology,John Bartram High School,NEIGHBORHOOD NETWORK 1,11,Not IEP,61,40,66%,16,26%,5,8%,0,0%,5,8%,0.356557377,1.42622947
4,4,101,101,JOHN BARTRAM HIGH SCHOOL,2013-2014,IEP,53,22,42%,District,NaN,NaN,NaN,Keystone,Literature,John Bartram High School,NEIGHBORHOOD NETWORK 1,11,IEP,28,20,71%,8,29%,0,0%,0,0%,0,0%,0.321428571,1.285714328


In [52]:
df3 = pd.merge(df_iep, df2, how='left', left_on='School_ID', right_on='School ID')
df3.head()

,Unnamed: 0,School_ID,School_Code,School Name_x,Cohort,Subgroup,Students,Grads,Rate,Governance_x,Unnamed: 8,Unnamed: 9,Unnamed: 10,Test Name,Subject,School Name_y,Learning Network,Grade,Category,Number Tested,Below Basic Count #,Below Basic Percent %,Basic Count #,Basic Percent %,Proficient Count #,Proficient Basic Percent %,Advanced Count #,Advanced Percent %,Prof and Adv Count #,Prof and Adv. Percent %,Average Score,Average Score2,School,School ID_old,School ID,Full Address,Enrollment,Rpt Type Long,Governance_y,Achievement_Score_Final,Progress_Score_Final,Climate_Score_Final,Overall_Score_Final,Latitude,Longitude,ASSAULTS,BOMB SCARE,BULLYING,BURGLARY,DISORDERLY CONDUCT,DRUG AND ALCOHOL OFFENSES,FIRE - INCENDIARY,GRAFFITI,HARASSMENT,MORALS OFFENSES,ROBBERY,SUICIDE,SUICIDE - ATTEMPTS AND THREATS,TAKE OFF,THEFT,THREAT,TRESPASSING,VANDALISM,WEAPONS,HARASSMENT_rate,ASSAULTS_rate,THREAT_rate,TAKE OFF_rate,DRUG AND ALCOHOL OFFENSES_rate,BURGLARY_rate,TRESPASSING_rate,BULLYING_rate,WEAPONS_rate,BOMB SCARE_rate,FIRE - INCENDIARY_rate,ROBBERY_rate,THEFT_rate,VANDALISM_rate,DISORDERLY CONDUCT_rate,SUICIDE_rate,MORALS OFFENSES_rate,GRAFFITI_rate,SUICIDE - ATTEMPTS AND THREATS_rate,SafetyIncidentRate,SafetyScore
0,0,101,101,JOHN BARTRAM HIGH SCHOOL,2013-2014,IEP,53,22,42%,District,NaN,NaN,NaN,Keystone,Algebra 1,John Bartram High School,NEIGHBORHOOD NETWORK 1,11,IEP,31,23,74%,8,26%,0,0%,0,0%,0,0%,0.314516129,1.258064508,John Bartram High School,101,101,"2401 S 67th St, Philadelphia, PA 19142",601.0,High School,District,0.0,28.0,11.0,14,39.92147,-75.234018,71.0,0.0,2.0,0.0,85.0,30.0,8.0,0.0,3.0,10.0,10.0,0.0,0.0,0.0,21.0,16.0,4.0,38.0,29.0,0.124792,2.953411,0.665557,0.0,1.24792,0.0,0.166389,0.083195,1.206323,0.0,0.332779,0.415973,0.873544,1.580699,3.535774,0.0,0.415973,0.0,0.0,5.657238,67.0
1,1,101,101,JOHN BARTRAM HIGH SCHOOL,2013-2014,IEP,53,22,42%,District,NaN,NaN,NaN,Keystone,Algebra 1,John Bartram High School,NEIGHBORHOOD NETWORK 1,11,Not IEP,63,30,48%,27,43%,6,10%,0,0%,6,10%,0.404761905,1.619047634,John Bartram High School,101,101,"2401 S 67th St, Philadelphia, PA 19142",601.0,High School,District,0.0,28.0,11.0,14,39.92147,-75.234018,71.0,0.0,2.0,0.0,85.0,30.0,8.0,0.0,3.0,10.0,10.0,0.0,0.0,0.0,21.0,16.0,4.0,38.0,29.0,0.124792,2.953411,0.665557,0.0,1.24792,0.0,0.166389,0.083195,1.206323,0.0,0.332779,0.415973,0.873544,1.580699,3.535774,0.0,0.415973,0.0,0.0,5.657238,67.0
2,2,101,101,JOHN BARTRAM HIGH SCHOOL,2013-2014,IEP,53,22,42%,District,NaN,NaN,NaN,Keystone,Biology,John Bartram High School,NEIGHBORHOOD NETWORK 1,11,IEP,28,24,86%,4,14%,0,0%,0,0%,0,0%,0.285714286,1.142857164,John Bartram High School,101,101,"2401 S 67th St, Philadelphia, PA 19142",601.0,High School,District,0.0,28.0,11.0,14,39.92147,-75.234018,71.0,0.0,2.0,0.0,85.0,30.0,8.0,0.0,3.0,10.0,10.0,0.0,0.0,0.0,21.0,16.0,4.0,38.0,29.0,0.124792,2.953411,0.665557,0.0,1.24792,0.0,0.166389,0.083195,1.206323,0.0,0.332779,0.415973,0.873544,1.580699,3.535774,0.0,0.415973,0.0,0.0,5.657238,67.0
3,3,101,101,JOHN BARTRAM HIGH SCHOOL,2013-2014,IEP,53,22,42%,District,NaN,NaN,NaN,Keystone,Biology,John Bartram High School,NEIGHBORHOOD NETWORK 1,11,Not IEP,61,40,66%,16,26%,5,8%,0,0%,5,8%,0.356557377,1.42622947,John Bartram High School,101,101,"2401 S 67th St, Philadelphia, PA 19142",601.0,High School,District,0.0,28.0,11.0,14,39.92147,-75.234018,71.0,0.0,2.0,0.0,85.0,30.0,8.0,0.0,3.0,10.0,10.0,0.0,0.0,0.0,21.0,16.0,4.0,38.0,29.0,0.124792,2.953411,0.665557,0.0,1.24792,0.0,0.166389,0.083195,1.206323,0.0,0.332779,0.415973,0.873544,1.580699,3.535774,0.0,0.415973,0.0,0.0,5.657238,67.0
4,4,101,101,JOHN BARTRAM HIGH SCHOOL,2013-2014,IEP,53,22,42%,District,NaN,NaN,NaN,Keystone,Literature,John Bartram High School,NEIGHBORHOOD NETWORK 1,11,IEP,28,20,71%,8,29%,0,0%,0,0%,0,0%,0.321428571,1.285714328,John Bartram High School,101,101,"2401 S 67th St, Philadelphia, PA 19142",601.0,High School,District,0.0,28.0,11.0,14,39.92147,-75.234018,71.0,0.0,2.0,0.0,85.0,30.0,8.0,0.0,3.0,10.0,10.0,0.0,0.0,0.0,21.0,16.0

# Export Data

In [53]:
# save data
df3.to_csv("Special-Education/IEP_data/integrated_data_iep.csv", sep=',', index=False)